In [1]:
import pandas as pd 

In [ ]:
df = pd.read_csv("../data/data1.csv")

In [3]:
df.head()


,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper


In [4]:
df.describe()

,conversation_id
count,188378.000000
mean,4313.102581
std,2491.096394
min,1.000000
25%,2156.000000
50%,4310.000000
75%,6469.000000
max,8628.000000


In [5]:
df["sentiment"].unique()

array([' Curious to dive deeper', ' Happy', ' Neutral', ' Surprised',
       ' Disgusted', ' Sad', ' Fearful', ' Angry'], dtype=object)

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens
    

In [10]:
df["processed_text"] = df['message'].apply(preprocess)

In [ ]:
df.to_csv("../data/cleaned_data.csv", index=False)  